In [ ]:
#!/cellar/users/btsui/anaconda2/bin/python
import param as param
import pandas as pd
import numpy as np
import os
CWD='/cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/Pipelines/snp/'
os.chdir(CWD)
allProcessedFnames=pd.Series(os.listdir(param.count_out_dir),dtype=np.str)
processed_runs=allProcessedFnames.str.extract('([DRS]RR\d+)',expand=False).values
print (processed_runs)
allDF=pd.DataFrame.from_csv(param.full_meta_dir)
#m1=True#
m_unprocessed=(~allDF.index.isin(processed_runs))

#m2=(allDF['pipe_line'].isin(param.supporting_seqs))
m2=allDF['LibraryStrategy'].isin(param.supporting_library_strategy)
m3=allDF['new_ScientificName'].isin(param.supporting_species)
m4=(allDF.Status=='live')
m5=(allDF.proj_accession_Visibility=='public')

#os.system(command)
#log_out_dir='/cellar/users/btsui/Data/SRA/all_seq/log/'
#count_out_dir='/cellar/users/btsui/Data/SRA/all_seq/chip/'


In [11]:
allDF['Bases']=pd.to_numeric(allDF['Bases'],errors='coerce')

In [12]:
#m6=(pd.to_numeric(allDF.Bases,errors='coerce')>(5*((10**6)*30)))
m6=(pd.to_numeric(allDF.Bases,errors='coerce')>(4*((10**6)*30)))

m7=allDF.LibraryLayout.isin(param.supporting_layouts)
m8=~allDF["Study"].isin(['ERP013950'])
m_valid_samples=m2&(m3)&m4&m5&m6&m7&m8
print ('# of valid samples data:',m_valid_samples.sum())
targetMetaDF=allDF[ m_unprocessed&m_valid_samples].sort_values('Bases')
targetMetaDF.to_csv(param.unprocessed_meta_dir)

command="qsub -t 1-"+str(targetMetaDF.shape[0])+" snp_all_seq.sh"
print (command)

# of valid samples data: 385016
qsub -t 1-108374 snp_all_seq.sh


In [15]:
os.system(command)

0

In [16]:
#!qdel 412089

In [24]:
!qstat -u btsui

job-ID  prior   name       user         state submit/start at     queue                          slots ja-task-ID 
-----------------------------------------------------------------------------------------------------------------
 411626 4.96275 QLOGIN     btsui        r     08/05/2018 09:40:27 all.q@nrnb-5-2.ucsd.edu            1        
 412090 4.99132 snp_all_se btsui        r     08/08/2018 10:21:22 long.q@nrnb-3-1.ucsd.edu           3 1
 412090 4.99132 snp_all_se btsui        r     08/08/2018 10:21:22 long.q@nrnb-2-1.ucsd.edu           3 2
 412090 4.99132 snp_all_se btsui        r     08/08/2018 10:21:22 long.q@nrnb-2-3.ucsd.edu           3 3
 412090 4.99132 snp_all_se btsui        r     08/08/2018 10:21:22 long.q@nrnb-3-5.ucsd.edu           3 4
 412090 4.99132 snp_all_se btsui        r     08/08/2018 10:21:22 long.q@nrnb-2-6.ucsd.edu           3 7
 412090 4.99132 snp_all_se btsui        r     08/08/2018 10:21:22 long.q@nrnb-3-1.ucsd.edu           3 8
 412090 4.99132 snp_all_se bts

In [28]:
!cat snp_all_seq.sh

#! /bin/bash
# Which shell to use
#$ -S /bin/bash
# Transfer all variables to job script (e.g. PATH, LD_LIBRARY_PATH, etc.)
#$ -V
# Queue to schedule jobs to
#$ -l h_rt=8:00:00
## -l u14
#$ -l h_vmem=2g
#$ -l long
#$ -l tmpfree=1T
#$ -l h=!(nrnb-5-0|nrnb-5-1|nrnb-5-2|nrnb-5-3|nrnb-5-4|nrnb-5-5|nrnb-5-6)
# Directory to send stdout and stderr
## -o /dev/null
## -e /dev/null
#$ -o /cellar/users/btsui/Data/sgeOut/
#$ -e /cellar/users/btsui/Data/sgeOut/
# Run in current working directory
#$ -cwd
# Array job 14175
#$ -t 1-6
## -p -0
#$ -r y
#$ -tc 20
#$ -pe smp 3
#SRR_FTP="/cellar/users/btsui/Data/SRA/META/processing.srr"
hostname
export PATH=/cellar/users/btsui/anaconda2/bin/:$PATH
/cellar/users/btsui/anaconda2/bin/python -u /cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/Pipelines/snp/run_one.py $SGE_TASK_ID


In [27]:
#!rm ./old_calculate_unprocessed.py